In [1]:
### criar um df principalmente a parte de inferir o schema que nao lembro
### window functions
### tratar nulos

In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


In [4]:
import sys
sys.path.append(r'/home/gabriel/Documents/proj/utils/')

from utils import createSparkSesion
from pyspark.sql import (functions as F,
                                        Window)
import matplotlib.pyplot as plt
import pandas as pd

spark esta no python 3.8.10

In [5]:
spark = createSparkSesion()

25/05/20 00:40:25 WARN Utils: Your hostname, GabrielUnbunto resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/05/20 00:40:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/05/20 00:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
spark

In [7]:
prefix_path = "/home/gabriel/Documents/local_datasets/raw/archive"
file        = "athletes.csv"

In [8]:
schema = StructType([
    StructField("id",               IntegerType(),  nullable=True),
    StructField("name",             StringType(),   nullable=True),
    StructField("gender",           StringType(),   nullable=True),
    StructField("age",              IntegerType(),  nullable=True),
    StructField("height",           IntegerType(),  nullable=True),
    StructField("weight",           IntegerType(),  nullable=True),
    StructField("team",             StringType(),   nullable=True),
    StructField("noc",              StringType(),   nullable=True),
    StructField("games",            StringType(),   nullable=True),
    StructField("year",             IntegerType(),  nullable=True),
    StructField("season",           StringType(),   nullable=True),
    StructField("city",             StringType(),   nullable=True),
    StructField("sport",            StringType(),   nullable=True),
    StructField("event",            StringType(),   nullable=True),
    StructField("medal",            StringType(),   nullable=True)
])

In [9]:
eda_df  = (spark.read
           .csv(f"{prefix_path}/{file}",
                  header=True, 
                  schema=schema)
           .withColumnRenamed("noc","team_prefix"))

In [10]:
eda_df.count()


271116

In [11]:
eda_df.takePandas()

,id,name,gender,age,height,weight,team,team_prefix,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA


<h1>Nomalization Diagram V1</h1>

<img src="/home/gabriel/Documents/proj/src/imgs/diagramas_athletes_history.png" width="700">


<h1>Creating normalized tables V1</h1>

In [12]:
from pyspark.sql.functions import monotonically_increasing_id

In [13]:
eda_df.takePandas()

,id,name,gender,age,height,weight,team,team_prefix,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA


In [14]:
gender = (
     eda_df
          .select(
               
             (F.when(F.col("gender")   == "M", "Male")
               .when(F.col("gender")   == "F", "Female")
               .otherwise("Nao cadastrado")
               ).alias("gender") 
          )
          .distinct()
          .withColumn("gender_id",monotonically_increasing_id())
          )

In [15]:
gender.takePandas()

,gender,gender_id
0,Female,0
1,Male,1


In [16]:
team = (
   eda_df
      .select(
         "team",
         "team_prefix"
      )
      .distinct()
      .withColumn("team_id",monotonically_increasing_id())
)

In [17]:
team.takePandas()

,team,team_prefix,team_id
0,Spain-1,ESP,0
1,Web III,DEN,1
2,Greece-3,GRE,2
3,Slovakia,SVK,3
4,Argonaut Rowing Club,CAN,4


In [27]:
athlete = (
   eda_df
      .select(
         "name",
         "team", # join drop
         "gender", #join drop
         "team_prefix", #join drop
         "height",
         "weight",
         "year",
         "age"
      )
      .distinct()
)


In [19]:
##mocking a birthday to replace the int age
## dedup first
## get the age less the year , getting some random month and day with a fixed seed and putting all together in a formated string , casting to date format
athlete_deduped_window = Window.partitionBy("name","team","gender").orderBy("age")

athlete_deduped = (
   athlete
      .withColumn("dedup_rn",F.row_number().over(athlete_deduped_window))
      .filter(F.col("dedup_rn") == 1)
      .drop("dedup_rn")
      .withColumn("day", F.expr("cast(floor(rand(42) * 25 + 1) as INT)"))
      .withColumn("month", F.expr("cast(floor(rand(42) * 12 + 1) as INT)"))
      .withColumn("birth",F.to_date(F.format_string("%04d-%02d-%02d",F.col("year"),F.col("month"),F.col("day"))))
      .select("name","team","team_prefix","gender","weight","height","birth")
      .withColumn("athlete_id",monotonically_increasing_id())
)


In [28]:
normalized_athlete = (
   athlete_deduped
      .join(team, on = ["team","team_prefix"], how = "inner")
      .withColumn("gender",F.when(F.col("gender")=="M","Male").otherwise("Female"))
      .join(gender, on = ["gender"])
      .select("athlete_id","gender_id","name","weight","height","birth","team_id")
)

In [ ]:
#final table to create the table on db
normalized_athlete.takePandas()

,athlete_id,gender_id,name,weight,height,birth,team_id
0,0,0,"""Aagje """"Ada"""" Kok (-van der Linden)""",85.0,183.0,1964-08-16,387
1,1,1,"""Abdoulaye """"Abdou"""" Sye Papa""",68.0,174.0,1960-07-13,17
2,2,1,"""Abram Dreyer """"Abe"""" Cohen""",90.0,190.0,1956-10-21,453
3,3,1,"""Adam Joseph """"AJ"""" Rosen""",80.0,188.0,2006-04-07,520
4,4,1,"""Adrian Daniel """"Adi"""" Popovici""",NaN,NaN,1992-09-17,104


In [21]:
athlete.count()
# to do ; ver quantidade de medalhas em cada intervalo de tempo com lag na parte da analise

189115

In [22]:
athlete_deduped.count()

139330

In [23]:
athlete_deduped.takePandas()

,name,team,team_prefix,gender,weight,height,birth,athlete_id
0,"""Aagje """"Ada"""" Kok (-van der Linden)""",Netherlands,NED,F,85.0,183.0,1964-08-16,0
1,"""Abdoulaye """"Abdou"""" Sye Papa""",France,FRA,M,68.0,174.0,1960-07-13,1
2,"""Abram Dreyer """"Abe"""" Cohen""",United States,USA,M,90.0,190.0,1956-10-21,2
3,"""Adam Joseph """"AJ"""" Rosen""",Great Britain,GBR,M,80.0,188.0,2006-04-07,3
4,"""Adrian Daniel """"Adi"""" Popovici""",Romania,ROU,M,NaN,NaN,1992-09-17,4


In [ ]:
athletes_history = (
   
)

In [24]:
eda_df.filter(F.col("team").rlike("\/")).takePandas(20)

,id,name,gender,age,height,weight,team,team_prefix,games,year,season,city,sport,event,medal
0,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
1,10875,Rodrigo Bertinotti,M,19.0,NaN,NaN,Pistoja/Firenze,ITA,1906 Summer,1906,Summer,Athina,Gymnastics,Gymnastics Men's Team All-Around,Bronze
2,12929,John Mary Pius Boland,M,25.0,NaN,NaN,Great Britain/Germany,GBR,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Doubles,Gold
3,19459,Emilio Cesarana,M,NaN,NaN,NaN,Barion/Bari-2,ITA,1906 Summer,1906,Summer,Athina,Rowing,Rowing Men's Coxed Pairs (1 kilometres),Silver
4,21717,Francesco Civera,M,23.0,NaN,NaN,Barion/Bari-2,ITA,1906 Summer,1906,Summer,Athina,Rowing,Rowing Men's Coxed Pairs (1 kilometres),Silver
5,21721,Cino Civinini,M,NaN,NaN,NaN,Pistoja/Firenze,ITA,1906 Summer,1906,Summer,Athina,Gymnastics,Gymnastics Men's Team All-Around,Bronze
6,26110,Basil Spalding de Garmendia,M,40.0,NaN,NaN,United States/France,USA,1900 Summer,1900,Summer,Paris,Tennis,Tennis Men's Doubles,Silver
7,26951,"""Maxime Omer Mathieu """"Max"""" Omer-Dcugis""",M,17.0,NaN,NaN,United States/France,FRA,1900 Summer,1900,Summer,Paris,Tennis,Tennis Men's Doubles,Silver
8,28174,Luigi Diana,M,40.0,NaN,NaN,Barion/Bari-2,ITA,1906 Summer,1906,Summer,Athina,Rowing,Rowing Men's Coxed Pairs (1 kilometres),Silver
9,29107,"""Hugh Lawrence """"Laurie"""" Doherty""",M,24.0,178.0,NaN,United States/Great Britain,GBR,1900 Summer,1900,Summer,Paris,Tennis,Tennis Mixed Doubles,Bronze
